## Dummy Dataset

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd 
import torch
import pickle


/home/jayhong7200/2025_MIMIC_IV/mimic_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
dummy_input_dict_path = Path('Results/Dummy/input_dict.pkl')
dummy_input_dict_path.parent.mkdir(parents=True, exist_ok=True)

if not dummy_input_dict_path.exists() :
    from transformers import AutoTokenizer

    text_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
    # Dummy
    age_dummy = torch.tensor([0])
    gender_dummy = torch.tensor([0])
    ethnicity_dummy = torch.tensor([0])
    types_dummy = torch.tensor([0]*23).unsqueeze(0)
    codes_dummy = torch.tensor([0]*23).unsqueeze(0)
    tabular_flag = torch.tensor([1])

    print(f"Age shape: {age_dummy.shape}")
    print(f"Gender shape: {gender_dummy.shape}")
    print(f"Ethnicity shape: {ethnicity_dummy.shape}")
    print(f"Types shape: {types_dummy.shape}")
    print(f"Codes shape: {codes_dummy.shape}")

    discharge = ""
    discharge_token = text_tokenizer(discharge, return_tensors="pt", padding=True, truncation=True, max_length=256) # 256 in the paper
    note_flag = torch.tensor([1])

    labvectors = torch.zeros(1, 114).unsqueeze(0)
    lab_flag = torch.tensor([1])

    label = torch.tensor([1])
    print(f"Discharge input_ids shape: {discharge_token['input_ids'].shape}")
    print(f"Discharge attention_mask shape: {discharge_token['attention_mask'].shape}")
    print(f"Labvectors shape: {labvectors.shape}")
    print(f"Label shape: {label.shape}")

    input_dict = {
        'age' : age_dummy,
        'gender' : gender_dummy,
        'ethnicity' : ethnicity_dummy,
        'types' : types_dummy,
        'codes' : codes_dummy,
        'tabular_flag' : tabular_flag,
        'discharge' : discharge_token,
        'note_flag' : note_flag,
        'labvectors' : labvectors,
        'lab_flag' : lab_flag,
        'label' : label    
    }

    with open(dummy_input_dict_path, 'wb') as f:
        import pickle
        pickle.dump(input_dict, f)  
    print(f"Dummy data saved to {dummy_input_dict_path}")
    
with open(dummy_input_dict_path, 'rb') as f:
    input_dict = pickle.load(f)  
print(f"Dummy data loaded from {dummy_input_dict_path}")

Dummy data loaded from Results/Dummy/input_dict.pkl


In [19]:
dummy_emb_dict_path = Path('Results/Dummy/emb_dict.pkl')
dummy_emb_dict_path.parent.mkdir(parents=True, exist_ok=True)

if not dummy_emb_dict_path.exists() :
    
    emb_dict = {
        'admission_id' : torch.zeros(1, 128),
        'tab_feature' : torch.zeros(1, 128),
        'tab_flag' : torch.zeros(1),
        'lab_feature' : torch.zeros(1, 128),
        'lab_flag' : torch.zeros(1),
        'note_feature' : torch.zeros(1, 128),
        'note_flag' : torch.zeros(1),
        'label' : torch.zeros(1),   
    }

    with open(dummy_emb_dict_path, 'wb') as f:
        import pickle
        pickle.dump(emb_dict, f)  
    print(f"Dummy data saved to {dummy_emb_dict_path}")
    
with open(dummy_emb_dict_path, 'rb') as f:
    emb_dict = pickle.load(f)  
print(f"Dummy data loaded from {dummy_emb_dict_path}")

Dummy data loaded from Results/Dummy/emb_dict.pkl


# Model

In [3]:
from src.dataset.tokenizer import MIMIC4Tokenizer
from src.utils.config import load_cfg
from argparse import Namespace
tokenizer = MIMIC4Tokenizer()
experiment = None

In [ ]:
# from src.core.e2e_mm_linear import Linear_Trainer
# # Tabular
# config_path = "configs/E2E_MM_mortality_90days_BS_512_hidden_128.yaml"
# config = load_cfg(config_path, args=Namespace(fusion_method='WeightedFusion'))
# model = Linear_Trainer(config, tokenizer, experiment).to('cuda')
# model = model.linear_backbone


# Sum WeightedFusion AttnMaskedFusion

In [28]:
from src.core.mm_linear import Linear_Trainer
# Tabular
config_path = "configs/MM_mortality_90days_BS_512_hidden_128.yaml"
config = load_cfg(config_path, args=Namespace(fusion_method='WeightedFusion'))
model = Linear_Trainer(config, experiment).to('cuda')
model = model.linear_backbone

Loading ETF Matrix from Results/ETF/ETF_128_IN_2_OUT_Seed_2026.pt


In [29]:
from thop import profile
# FLOPS
model.eval()
try : 
    macs, params = profile(model, inputs=input_dict, verbose=False)
except : 
    macs, params = profile(model, inputs=emb_dict, verbose=False)
flops = macs * 2
print("MACs:", macs, "FLOPs:", flops, "Params:", params)

# conver to GB
flops_mb = flops / 1e6
params_mb = params / 1e6
print(f"FLOPs (in MFLOPs): {flops_mb:.2f}")
print(f"Params (in M): {params_mb:.2f}")


print(macs, flops, params, round(flops_mb,2), round(params_mb,2))

MACs: 0.0 FLOPs: 0.0 Params: 0
FLOPs (in MFLOPs): 0.00
Params (in M): 0.00
0.0 0.0 0 0.0 0.0


In [2]:
import pandas as pd
"""
                        MACs	        FLOPs	        Params	        FLOPs (in MFLOPs)	Params (in M)
tabular	                11368448.0	    22736896.0	    510464.0	    20.00	            0.51
lab	                    210944.0	    421888.0	    210688.0	    0.42	            0.21
note	                824320.0	    1648640.0	    429824.0	    1.65	            0.43
E2E_SUM	                12420096.0	    24840192.0	    1167488.0	    24.84	            1.17
E2E_WeightedFusion      12403712.0      24807424.0      1150976.0       24.80               1.15
E2E_Attn                12453256.0      24906512.0      1200643.0       24.90               1.20
SUM                     32768.0         65536.0         16512.0         0.07                0.02
WeightedFusion          0               0               0.0             0.0                 0.0
AttnMaskedFusion        99088.0         198176.0        49667.0         0.2                 0.05
MUSE	                83533056.0	    167066112.0	    1391105.0	    167.07	            1.39
"""

result_df = [
    ['tabular', 11368448, 22736896, 510464, 20.00, 0.51],
    ['lab', 210944, 421888, 210688, 0.42, 0.21],
    ['note', 824320, 1648640, 429824, 1.65, 0.43],
    ['E2E_SUM', 12420096, 24840192, 1167488, 24.84, 1.17],
    ['E2E_WeightedFusion', 12403712, 24807424, 1150976, 24.80, 1.15],
    ['E2E_Attn', 12453256, 24906512, 1200643, 24.90, 1.20],
    ['MUSE', 83533056, 167066112, 1391105, 167.07, 1.39],
    ['SUM', 32768, 65536, 16512, 0.07, 0.02],
    ['WeightedFusion', 0, 0, 0, 0.0, 0.0],
    ['AttnMaskedFusion', 99088, 198176, 49667, 0.2, 0.05],
]

result_df = pd.DataFrame(result_df, columns=['Model', 'MACs', 'FLOPs', 'Params', 'FLOPs (in MFLOPs)', 'Params (in M)'])
result_df

,Model,MACs,FLOPs,Params,FLOPs (in MFLOPs),Params (in M)
0,tabular,11368448,22736896,510464,20.00,0.51
1,lab,210944,421888,210688,0.42,0.21
2,note,824320,1648640,429824,1.65,0.43
3,E2E_SUM,12420096,24840192,1167488,24.84,1.17
4,E2E_WeightedFusion,12403712,24807424,1150976,24.80,1.15
5,E2E_Attn,12453256,24906512,1200643,24.90,1.20
6,MUSE,83533056,167066112,1391105,167.07,1.39
7,SUM,32768,65536,16512,0.07,0.02
8,WeightedFusion,0,0,0,0.00,0.00
9,AttnMaskedFusion,99088,198176,49667,0.20,0.05
